In [24]:
from tools_basic import tools
from dotenv import load_dotenv

load_dotenv(".env")

True

In [25]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain_anthropic import ChatAnthropic
from langchain_community.chat_models import ChatPerplexity
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_cohere import ChatCohere
from langchain_core.runnables import ConfigurableField

In [26]:
llm = ChatAnthropic(
    model="claude-3-opus-20240229",
    # max_tokens=,
    temperature=0.9,
    # anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY", "not_provided"),
    streaming=True,
    verbose=True,
).configurable_alternatives(  # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    default_key="anthropic_claude_3_opus",
    openai_gpt_3_5_turbo_1106=ChatOpenAI(
        model="gpt-3.5-turbo-1106",
        verbose=True,
        streaming=True,
        temperature=0.9,
    ),
    openai_gpt_4_turbo_preview=ChatOpenAI(
        temperature=0.9,
        model="gpt-4-turbo-preview",
        verbose=True,
        streaming=True,
    ),
    pplx_sonar_medium_chat=ChatPerplexity(
        model="sonar-medium-chat", temperature=0.9, verbose=True, streaming=True
    ),
    mistral_large=ChatMistralAI(
        model="mistral-large-latest", temperature=0.9, verbose=True, streaming=True
    ),
    command_r_plus=ChatCohere(
        model="command-r-plus", temperature=0.9, verbose=True, streaming=True
    ),
)
llm_with_tools = llm.bind_tools(tools=tools)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
# llm_with_tools.with_config(
#     {
#         "llm": "openai_gpt_4_turbo_preview",
#     }
# )
agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
    )
    | prompt
    | llm_with_tools
    | ToolsAgentOutputParser()
)

In [27]:
agent.with_config(
    {
        "llm": "openai_gpt_4_turbo_preview",
    }
)
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
streams = executor.astream_events({"input": "今天有什么关于btc的新闻？"}, version="v1")
async for chunk in streams:
    print(chunk)



> Entering new AgentExecutor chain...{'event': 'on_chain_start', 'run_id': '1b27d6a2-6ac3-44e4-9bea-6ecda996dc4a', 'name': 'AgentExecutor', 'tags': [], 'metadata': {}, 'data': {'input': {'input': '今天有什么关于btc的新闻？'}}}

{'event': 'on_chain_start', 'name': 'RunnableSequence', 'run_id': '752941dc-c266-417d-8867-b5356878fc06', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': '5303633b-3317-4076-9ed4-0034e8159f51', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {}}
{'event': 'on_chain_stream', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': '5303633b-3317-4076-9ed4-0034e8159f51', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'chunk': {'input': '今天有什么关于btc的新闻？', 'intermediate_steps': []}}}
{'event': 'on_chain_start', 'name': 'RunnableParallel<agent_scratchpad>', 'run_id': 'e1ed6538-48c1-411c-8903-9550775c2a19', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableLambda', 'run_i

/Users/zhangleping/github.com/holynull/chat-langchain/.venv/lib/python3.10/site-packages/langchain_anthropic/chat_models.py:442: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")
/Users/zhangleping/github.com/holynull/chat-langchain/.venv/lib/python3.10/site-packages/langchain_anthropic/chat_models.py:527: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn(


{'event': 'on_chat_model_stream', 'name': 'ChatAnthropic', 'run_id': 'be0aebb2-3033-48ad-8364-fcfb6a925db2', 'tags': ['seq:step:3'], 'metadata': {}, 'data': {'chunk': AIMessageChunk(content=[{'text': '<thinking>\n根据用户的问题，他们想要知道今天关于比特币（BTC）的新闻。使用ArxivSearch来搜索科学论文可能不太相关，因为用户问的是最新的新闻，而不是研究论文。  \n\n要回答这个问题，最相关的工具是search函数，将search_type参数设置为"news"，这样就可以专门搜索新闻结果。query参数可以设置为"btc news"。search函数需要的所有参数都有了，可以调用它了。\n</thinking>', 'type': 'text'}, {'id': 'toolu_01GhefWhod8Jner6q8e619FR', 'input': {'query': 'btc news', 'search_type': 'news'}, 'name': 'search', 'type': 'tool_use'}], id='run-be0aebb2-3033-48ad-8364-fcfb6a925db2', tool_calls=[{'name': 'search', 'args': {'query': 'btc news', 'search_type': 'news'}, 'id': 'toolu_01GhefWhod8Jner6q8e619FR'}], tool_call_chunks=[{'name': 'search', 'args': '{"query": "btc news", "search_type": "news"}', 'id': 'toolu_01GhefWhod8Jner6q8e619FR', 'index': 0}])}}
{'event': 'on_chat_model_end', 'name': 'ChatAnthropic', 'run_id': 'be0aebb2-3033-48ad-8364-fcfb6a925

/Users/zhangleping/github.com/holynull/chat-langchain/.venv/lib/python3.10/site-packages/langchain_anthropic/chat_models.py:442: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")
/Users/zhangleping/github.com/holynull/chat-langchain/.venv/lib/python3.10/site-packages/langchain_anthropic/chat_models.py:527: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn(


{'event': 'on_chat_model_stream', 'name': 'ChatAnthropic', 'run_id': '19a9192c-b0da-4731-bf94-8340bef42cc9', 'tags': ['seq:step:3'], 'metadata': {}, 'data': {'chunk': AIMessageChunk(content='根据搜索结果，今天有几个关于比特币（BTC）的重要新闻：\n\n1. 香港现货比特币ETF正式开始交易，这标志着该领域的一个里程碑时刻。\n\n2. 有分析师预测5月份有3个看涨的山寨币将超过比特币的表现。 \n\n3. 比特币矿工的盈利能力下降，挖矿难度调整后，哈希价格创下历史新低。\n\n4. MicroStrategy公司继续购买比特币，目前持有价值近140亿美元的BTC。\n\n5. Samourai钱包联合创始人Keonne Rodriguez不认罪，以100万美元保释。\n\n6. 一位加密行业知名人士Angelo Babb预测2024年第二季度3种加密货币将创历史新高。\n\n所以总的来说，虽然比特币矿工利润下降，但机构投资者如MicroStrategy仍在持续买入BTC，香港推出现货ETF也有利好作用。市场情绪整体偏乐观，预计未来比特币和部分山寨币可能创新高。', id='run-19a9192c-b0da-4731-bf94-8340bef42cc9')}}
{'event': 'on_chat_model_end', 'name': 'ChatAnthropic', 'run_id': '19a9192c-b0da-4731-bf94-8340bef42cc9', 'tags': ['seq:step:3'], 'metadata': {}, 'data': {'input': {'messages': [[SystemMessage(content='You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.'), HumanMessage(content='今天有什么关于btc的新闻？'), AIMessageChunk(conte